In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
ls

 2023.05.17.541168v3.full.pdf            O1_data_anatomy/
'Analyze connectivity reach.ipynb'       O1_data_anatomy.xz
'Analyze connectivity strengths.ipynb'   O1_data_physiology/
 Circuitexamples.ipynb                   O1_data_physiology_withfix.xz
'Circuit exploration examples.ipynb'     save_new_positions_3Dplots.ipynb
 import_3Dplots_rearange.ipynb           SpontaneousSimulations/
 import_h5infos.ipynb*                  'Synaptic innervation patterns.ipynb'
 import_sonatafiles.ipynb                topology_analysis.log
 node_sets.json                          voxel_atlas_data/
 nodes_new.h5


In [3]:
import os
import bluepysnap  # https://bluebrainsnap.readthedocs.io/en/stable/
import neurom  # https://neurom.readthedocs.io/en/stable/
import pandas
import numpy
import scipy

from neurom import viewer

# fn_extr = "/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_anatomy/circuit_config.json"
fn_extr = "O1_data_anatomy/circuit_config.json"
circ_extr = bluepysnap.Circuit(fn_extr)


### Neuron populations and their properties

The model contains a number of neuron (or node) populations. Let's list them, then I'll explain them

In [4]:
print(circ_extr.nodes.population_names)

['POm', 'S1nonbarrel_neurons', 'VPM', 'external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical', 'external_midrange__S1nonbarrel_neurons__chemical']


We find five populations. One population is the most important one: 'S1nonbarrel_neurons' denotes the neurons actually in the model. The ones that can be simulated. The other four denote external populations that innervate the 'S1nonbarrel_neurons':

'POm' and 'VPM' are innervating neurons from those thalamic nuclei respectively. Representing a "matrix-type" and "core-type" projection respectively.
'external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical' and 'external_midrange__S1nonbarrel_neurons__chemical' denote nonbarrel S1 neurons outside the modeled population that innervate the modeled population. The difference between them is as follows: 'external_midrange...' innervates via long-range connections, while 'external_S1...' innervates via local connections. For an explanation of "local" vs. "long-range" connections, see the accompanying manuscript. 


Now we can load the properties of the neurons in these populations.

In [5]:
lst_properties = ['layer', 'region', 'synapse_class', 'population',
                  'orientation_x', 'orientation_y', 'orientation_z', 'orientation_w',
                  'morph_class', 'model_type', 'mtype', 'morphology',
                  'x', 'y', 'z']
n_props = circ_extr.nodes.get(properties=lst_properties)
display(n_props.loc['S1nonbarrel_neurons'])

AttributeError: 'generator' object has no attribute 'loc'

In [6]:
lst_properties = ['layer']
n_props = circ_extr.nodes.get(properties=lst_properties)

In [7]:
n_props

<generator object NetworkObject.get at 0x7f606fb3ef90>

'layer', 'region' specify in which cortical region and layer a neuron was placed.

'x', 'y', 'z' specify the exact coordinate (in um) in the atlas that a neuron was placed at.

'orientation_x', 'orientation_y', 'orientation_z', 'orientation_w' define a quarternion that specifies the rotation that is applied to the neuron morphology in placement.

'synapse_class', 'morph_class', 'mtype' specify the class of neuron in terms of being excitatory vs inhibitory, pyramidal vs interneuron and the morphological type.

'morphology' specifies the name of the morphological reconstruction of the neuron

'model_type' specifies whether a neuron is actually part of the model that can be simulated ("biophysical") or an external innervator ("virtual"). As explained above, all 'S1nonbarrel_neurons' are "biophysical", the others are all "virtual".


Note that for the "virtual" neurons some properties are unassigned and therefore reported as NaN. 

In [14]:
display(n_props.loc['external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical'])  # Some columns are NaN

AttributeError: 'generator' object has no attribute 'loc'

### Neuron morphologies

For the 'S1nonbarrel_neurons' population, we can then load their morphologies with some simple helper functions.

For this example, we load and draw and exemplary neuron. For more that can be done with the loaded morphology, see the NeuroM documentation linked in the first cell of this notebook.

In [21]:
S1nonbarrel_cfg = circ_extr.config["networks"]["nodes"][0]["populations"]["S1nonbarrel_neurons"]

def transform_neuron(nrn_morph, neuron_row):
    rot = scipy.spatial.transform.Rotation.from_quat(neuron_row[["orientation_x", "orientation_y",
                                                                 "orientation_z", "orientation_w"]].values)
    rot = neurom.geom.transform.Rotation(rot.as_matrix())
    tl = neurom.geom.transform.Translation(neuron_row[["x", "y", "z"]].values)
    return nrn_morph.transform(rot).transform(tl)
    
def load_neuron(population_cfg, neuron_row, transform=True):
    assert neuron_row["model_type"] == "biophysical", "Cannot load morphologies of virtual neurons"
    fn = os.path.join(
        population_cfg["alternate_morphologies"]["neurolucida-asc"],
        neuron_row["morphology"] + ".asc"
    )
    nrn = neurom.load_neuron(fn)
    if transform:
        nrn = transform_neuron(nrn, neuron_row)
    return nrn

nrn_morph = load_neuron(S1nonbarrel_cfg, n_props.loc[('S1nonbarrel_neurons', 0)])
viewer.draw(nrn_morph)

AttributeError: 'generator' object has no attribute 'loc'

### Synaptic connectivity

The synaptic connectivity of the model is accessed using circ_extr.edges.
As an example, let's find out which neurons innervate an exemplary neuron

In [22]:
# The method we use below requires node ids as input. So we load the ids of the "S1nonbarrel_neurons"
node_ids = circ_extr.nodes.ids().filter_population("S1nonbarrel_neurons")

# This will be our examplary neuron. Let's look at its properties
display(n_props.loc[node_ids[0]])

# Which neurons are afferent to the first one?
aff = circ_extr.edges.afferent_nodes(node_ids[0])
# What's returned can be used as index into the neuron properties data frame
aff_props = n_props.loc[aff]

# Look at the regions of neurons innervating the exemplary one from within the model population
display(aff_props["region"]["S1nonbarrel_neurons"].value_counts())

AttributeError: 'generator' object has no attribute 'loc'

We see that the exemplary neuron is a L1_DAC neuron in the S1FL (front limb) area. From within the modeled population, it is mostly innervated by other S1FL neurons.

Let's see what external neurons innervate it.

In [6]:
# External innervation via local connections
display(aff_props["region"]["external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].value_counts())

# External innervation via long-range connections
display(aff_props["region"]["external_midrange__S1nonbarrel_neurons__chemical"].value_counts())


S1FL     37
S1DZ      9
S1J       8
S1ULp     6
Name: region, dtype: int64

S1ULp    135
S1HL      84
S1FL      43
S1DZ      37
S1J       36
S1Tr      36
S1DZO      6
S1Sh       5
Name: region, dtype: int64

Unsurprisingly, we see that external innervation via local connection comes also mostly from S1FL. For long-range connections though, S1ULp is dominating.

Properties of the synapses (such as their dendritic and axon locations) can be accessed as follows:

In [10]:
circ_extr.edges.afferent_edges(node_ids[0], properties=['afferent_section_id',
                                                        'afferent_segment_id', 
                                                        'afferent_segment_offset'])

afferent_section_id  \
population                                       edge_ids                       
POm__S1nonbarrel_neurons__chemical               0                        208   
                                                 1                        208   
                                                 2                        201   
                                                 3                        201   
                                                 4                        212   
...                                                                       ...   
external_midrange__S1nonbarrel_neurons__chemical 543                      176   
                                                 544                      178   
                                                 545                      185   
                                                 546                      185   
                                                 547                      182   

                                                          afferent_segment_id  \
population                                       edge_ids                       
POm__S1nonbarrel_neurons__chemical               0                         96   
                                                 1                         47   
                                                 2                        107   
                                                 3                        100   
                                                 4                         97   
...                                                                       ...   
external_midrange__S1nonbarrel_neurons__chemical 543                        8   
                                                 544                       78   
                                                 545                       71   
                                                 546                      146   
                                                 547                        9   

                                                          afferent_segment_offset  
population                                       edge_ids                          
POm__S1nonbarrel_neurons__chemical               0                       0.687199  
                                                 1                       0.731116  
                                                 2                       3.612747  
                                                 3                       1.182481  
                                                 4                        0.85173  
...                                                                           ...  
external_midrange__S1nonbarrel_neurons__chemical 543                     1.091046  
                                                 544                     0.093693  
                                                 545                     0.189781  
                                                 546                      0.38004  
                                                 547                     1.365389  

[1179 rows x 3 columns]

These were just simple examples. For more analyses on the connectivity, see the SONATA documentation, and the bluepysnap documentation linked in the first cell.